# Data cleaning
We want the class with labels as numerical value and the body with clean text.

This will remove:
* duplicates
* NaN entires
* non english
* url, html

* make it lowercase
* combine title and body

In [1]:
import pandas as pd
import sys
import numpy as np
sys.path.append("../../scripts_shared/")
from preprocess_text import preprocess_text


In [2]:
# Read CSV into a dataframe
#high_priority = pd.read_csv("csv/high_priority_no_td.csv", index_col=0)
#medium_priority = pd.read_csv("csv/medium_priority_no_td.csv", index_col=0)
low_priority = pd.read_csv("csv/low_priority_no_td.csv", index_col=0)
not_priority = pd.read_csv("csv/not_priority.csv", index_col=0)

/tmp/ipykernel_38653/220004500.py:5: DtypeWarning: Columns (0,2,3,4,5,6,7,8,9) have mixed types. Specify dtype option on import or set low_memory=False.
  not_priority = pd.read_csv("csv/not_priority.csv", index_col=0)


In [3]:
# Number or different labels
low_priority.labels.value_counts().to_frame()[:50]

,count
labels,
enhancement low priority,5241
low priority,5169
bug low priority,2662
Low Priority,2584
pri:low pri:lowest,2565
Priority: Low,2377
priority: low,1126
bug priority: low,1093
Priority: Low Type: Enhancement,1093


In [4]:
not_priority.labels.value_counts().to_frame()[:50]

,count
labels,
stale-branch,6827422
stale branch 🗑️,348533
enhancement,240489
bug,237012
security vulnerability,52497
question,34359
feature,20451
documentation,17632
Bug,12098


In [5]:
contains_stale = not_priority['labels'].str.contains("stale", case=False, na=False)
not_priority= not_priority[~contains_stale].reset_index(drop=True)
not_priority.reset_index(drop=True, inplace=True)
not_priority.labels.value_counts().to_frame()[:50]

,count
labels,
enhancement,240489
bug,237012
security vulnerability,52497
question,34359
feature,20451
documentation,17632
Bug,12098
help wanted,10727
wontfix,7180


In [6]:
#Give each priority a label by number.
# 'Label encoding'. Makes is easier for machine learning models to work with categorical data.
low_priority["labels"] = 0
low_priority["class"] = "low_priority"
not_priority["labels"] = 1
not_priority["class"] = "not_priority"

In [7]:
priority = pd.concat([low_priority, not_priority] , ignore_index = True)

In [8]:
priority[priority["repo"] == "python/mypy"]

,id,type,created_at,repo,repo_url,action,title,labels,body,class
210262,1.142706e+10,IssuesEvent,2020-02-03 23:33:27,python/mypy,https://api.github.com/repos/python/mypy,closed,Type of conditional expression is object,1,The type of the following conditional expressi...,not_priority
210291,1.142707e+10,IssuesEvent,2020-02-03 23:35:24,python/mypy,https://api.github.com/repos/python/mypy,closed,Type inference of Tuples returns object instead,1,While interacting with `zip` I encountered a v...,not_priority
225547,1.362788e+10,IssuesEvent,2020-09-24 13:13:03,python/mypy,https://api.github.com/repos/python/mypy,opened,Daemon support for --follow-imports=silent,1,"After #5870 is done, it would be nice to also ...",not_priority
234003,1.363241e+10,IssuesEvent,2020-09-24 19:36:27,python/mypy,https://api.github.com/repos/python/mypy,opened,regression: assignment of 'builtins.type' now ...,1,\r\n**Bug Report**\r\n\r\n#7963 causes a serio...,not_priority
251028,2.407484e+10,IssuesEvent,2022-09-18 17:03:35,python/mypy,https://api.github.com/repos/python/mypy,opened,stubtest: more concise error for forgotten arg...,1,**Feature**\r\n\r\nWhen the implementation add...,not_priority
...,...,...,...,...,...,...,...,...,...,...
9377824,7.349652e+09,IssuesEvent,2018-03-08 11:27:27,python/mypy,https://api.github.com/repos/python/mypy,closed,Legal Python circular imports result in loss o...,1,Example:\r\n\r\n`test/__init__.py`:\r\n```pyth...,not_priority
9377825,7.349652e+09,IssuesEvent,2018-03-08 11:27:27,python/mypy,https://api.github.com/repos/python/mypy,closed,Crash when serializing property with forward r...,1,This serialization test case fails with a cras...,not_priority
9378600,7.349972e+09,IssuesEvent,2018-03-08 12:44:13,python/mypy,https://api.github.com/repos/python/mypy,opened,Suggest type annotations for partial types spa...,1,We don't support inferring partial types from ...,not_priority
9378613,7.349979e+09,IssuesEvent,2018-03-08 12:45:41,python/mypy,https://api.github.com/repos/python/mypy,closed,What to do about inferring types from multiple...,1,Typically mypy infers the type of the variable...,not_priority


In [9]:
# Remove mypy from the dataset
priority = priority[priority["repo"] != "python/mypy"]
priority

,id,type,created_at,repo,repo_url,action,title,labels,body,class
0,1.141755e+10,IssuesEvent,2020-02-03 00:02:50,poissonconsulting/term,https://api.github.com/repos/poissonconsulting...,opened,Document levels of validation in vld_term(),0,As this is what other functions reference.,low_priority
1,1.141766e+10,IssuesEvent,2020-02-03 00:46:42,WordPress/gutenberg,https://api.github.com/repos/WordPress/gutenberg,closed,Z-index issue using Classic block,0,Selecting a link in the classic block and havi...,low_priority
2,1.141777e+10,IssuesEvent,2020-02-03 01:25:06,MarkBind/markbind,https://api.github.com/repos/MarkBind/markbind,reopened,Indicate the language of a code block,0,**Tell us about your environment**\r\n\r\n* **...,low_priority
3,1.141780e+10,IssuesEvent,2020-02-03 01:34:56,MarkBind/markbind,https://api.github.com/repos/MarkBind/markbind,closed,"Use one attribute name for header, title, name",0,May be better to choose one of the three and u...,low_priority
4,1.141786e+10,IssuesEvent,2020-02-03 01:52:04,poissonconsulting/term,https://api.github.com/repos/poissonconsulting...,closed,Document levels of validation in vld_term(),0,As this is what other functions reference.,low_priority
...,...,...,...,...,...,...,...,...,...,...
9396571,8.109961e+09,IssuesEvent,2018-08-14 09:21:15,urbit/arvo,https://api.github.com/repos/urbit/arvo,opened,%bad-text trips up hall JSON conversion,1,"Haven't tested this in detail yet, but pretty ...",not_priority
9396572,8.109961e+09,IssuesEvent,2018-08-14 09:21:16,highcharts/highcharts-react,https://api.github.com/repos/highcharts/highch...,closed,HighMaps mapBubble type,1,"Hello,\r\nIt is possible to create also HighMa...",not_priority
9396573,8.109982e+09,IssuesEvent,2018-08-14 09:25:07,Loriowar/comindivion,https://api.github.com/repos/Loriowar/comindivion,opened,Add a validation on a belonging of a predicate...,1,Now user can change html content of the intera...,not_priority
9396574,8.109983e+09,IssuesEvent,2018-08-14 09:25:24,TiiQu-Network/TQ-test-page,https://api.github.com/repos/TiiQu-Network/TQ-...,closed,Error on submit with no values,1,TypeError: Too few arguments in function sum (...,not_priority


In [10]:
# Drop duplicates by the content of the title
priority = priority.drop_duplicates(subset=['title'], keep='last')
priority.dropna(inplace=True)
priority.reset_index(inplace=True)
priority.drop(columns=["index"] , inplace= True)
priority["class"].value_counts()

/tmp/ipykernel_38653/1842014761.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  priority.dropna(inplace=True)
/tmp/ipykernel_38653/1842014761.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  priority.drop(columns=["index"] , inplace= True)


class
not_priority    1330864
low_priority     135000
Name: count, dtype: int64

In [11]:
# Split the dataframe 
low_priority = priority.loc[priority['class'] == 'low_priority'].copy()
not_priority = priority.loc[priority['class'] == 'not_priority'].copy()

In [12]:
low_label_counts = low_priority["class"].value_counts()
not_label_counts = not_priority["class"].value_counts()
lp_count = low_label_counts["low_priority"]
print(f"low count: {lp_count}")
not_count = not_label_counts["not_priority"]
print(f"not count: {not_count}")

low count: 135000
not count: 1330864


In [13]:
not_priority = not_priority.sample(frac=lp_count/not_count, random_state=42)
low_vs_not = pd.concat([low_priority, not_priority] , ignore_index = True)
low_vs_not["class"].value_counts()

class
low_priority    135000
not_priority    135000
Name: count, dtype: int64

In [14]:
print(low_vs_not["title"][0])
print(low_vs_not["body"][0])

Z-index issue using Classic block
Selecting a link in the classic block and having it active then selecting another link in the left sidebar it looked like this:

![screen shot 2018-05-31 at 18 37 33](https://user-images.githubusercontent.com/5323259/40795551-b05f6796-6502-11e8-8510-65198a4cf7df.png)



In [15]:
# Copy content of body to a new col named text
low_vs_not["text"] = low_vs_not["title"] + low_vs_not["body"]
low_vs_not.tail()

,id,type,created_at,repo,repo_url,action,title,labels,body,class,text
269995,1.576438e+10,IssuesEvent,2021-03-31 13:11:25,GastonGit/Hot-Twitch-Clips,https://api.github.com/repos/GastonGit/Hot-Twi...,closed,Twitch client needs to be able to reconnect,1,"Needs to be set as option in ""options"" argumen...",not_priority,Twitch client needs to be able to reconnectNee...
269996,1.516093e+10,IssuesEvent,2021-02-12 08:07:00,bryntum/support,https://api.github.com/repos/bryntum/support,closed,Icon is not shown for resource time ranges whe...,1,https://www.bryntum.com/forum/viewtopic.php?f=...,not_priority,Icon is not shown for resource time ranges whe...
269997,1.309422e+10,IssuesEvent,2020-08-03 11:59:27,iobio/clin.iobio,https://api.github.com/repos/iobio/clin.iobio,closed,Update Frameshift or Mosaic logos on pdf report,1,![image](https://user-images.githubusercontent...,not_priority,Update Frameshift or Mosaic logos on pdf repor...
269998,9.816864e+09,IssuesEvent,2019-06-13 15:28:47,gotthardp/python-mercuryapi,https://api.github.com/repos/gotthardp/python-...,closed,Read_count staying at 1,1,"Hello, When I run continous read mode the read...",not_priority,"Read_count staying at 1 Hello, When I run cont..."
269999,2.661149e+10,IssuesEvent,2023-01-24 00:49:12,thomasjacquin/allsky,https://api.github.com/repos/thomasjacquin/allsky,closed,*** WARNING: time to take image () differs fro...,1,<!-- ==================== Section 1 ==========...,not_priority,*** WARNING: time to take image () differs fro...


In [16]:
low_vs_not["text"][0]

'Z-index issue using Classic blockSelecting a link in the classic block and having it active then selecting another link in the left sidebar it looked like this:\r\n\r\n![screen shot 2018-05-31 at 18 37 33](https://user-images.githubusercontent.com/5323259/40795551-b05f6796-6502-11e8-8510-65198a4cf7df.png)\r\n'

In [17]:
# Make a new dataframe with only text, label and class cols.
all_priority_subset = low_vs_not[["text" , "labels" , "class"]]
all_priority_subset

,text,labels,class
0,Z-index issue using Classic blockSelecting a l...,0,low_priority
1,Indicate the language of a code block**Tell us...,0,low_priority
2,"Use one attribute name for header, title, name...",0,low_priority
3,Document levels of validation in vld_term()As ...,0,low_priority
4,Support puml files in boilerplatessee [this co...,0,low_priority
...,...,...,...
269995,Twitch client needs to be able to reconnectNee...,1,not_priority
269996,Icon is not shown for resource time ranges whe...,1,not_priority
269997,Update Frameshift or Mosaic logos on pdf repor...,1,not_priority
269998,"Read_count staying at 1 Hello, When I run cont...",1,not_priority


In [18]:
# Convert to string
all_priority_subset["text_str"] = all_priority_subset['text'].astype(str)

/tmp/ipykernel_38653/3106036705.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  all_priority_subset["text_str"] = all_priority_subset['text'].astype(str)


In [19]:
all_priority_subset

,text,labels,class,text_str
0,Z-index issue using Classic blockSelecting a l...,0,low_priority,Z-index issue using Classic blockSelecting a l...
1,Indicate the language of a code block**Tell us...,0,low_priority,Indicate the language of a code block**Tell us...
2,"Use one attribute name for header, title, name...",0,low_priority,"Use one attribute name for header, title, name..."
3,Document levels of validation in vld_term()As ...,0,low_priority,Document levels of validation in vld_term()As ...
4,Support puml files in boilerplatessee [this co...,0,low_priority,Support puml files in boilerplatessee [this co...
...,...,...,...,...
269995,Twitch client needs to be able to reconnectNee...,1,not_priority,Twitch client needs to be able to reconnectNee...
269996,Icon is not shown for resource time ranges whe...,1,not_priority,Icon is not shown for resource time ranges whe...
269997,Update Frameshift or Mosaic logos on pdf repor...,1,not_priority,Update Frameshift or Mosaic logos on pdf repor...
269998,"Read_count staying at 1 Hello, When I run cont...",1,not_priority,"Read_count staying at 1 Hello, When I run cont..."


In [20]:
# Clean the data.
all_priority_subset["text_clean"] = all_priority_subset["text_str"].map(preprocess_text)

/tmp/ipykernel_38653/63294665.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  all_priority_subset["text_clean"] = all_priority_subset["text_str"].map(preprocess_text)


In [21]:
# Make a subset with text_clean and label
priority_label_text = all_priority_subset[["text_clean" , "labels", "class"]]
priority_label_text

,text_clean,labels,class
0,zindex issue using classic blockselecting a li...,0,low_priority
1,indicate the language of a code blocktell us a...,0,low_priority
2,use one attribute name for header title namema...,0,low_priority
3,document levels of validation in vldtermas thi...,0,low_priority
4,support puml files in boilerplatessee this com...,0,low_priority
...,...,...,...
269995,twitch client needs to be able to reconnectnee...,1,not_priority
269996,icon is not shown for resource time ranges whe...,1,not_priority
269997,update frameshift or mosaic logos on pdf repor...,1,not_priority
269998,readcount staying at 1 hello when i run contin...,1,not_priority


In [22]:
# Need to dropna here since cleaning function returns NaN for not english text.
priority_label_text.dropna(inplace=True)
priority_label_text.reset_index(inplace=True)
priority_label_text.drop(columns=["index"] , inplace= True)

priority_label_text

/tmp/ipykernel_38653/239147930.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  priority_label_text.dropna(inplace=True)
/tmp/ipykernel_38653/239147930.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  priority_label_text.drop(columns=["index"] , inplace= True)


,text_clean,labels,class
0,zindex issue using classic blockselecting a li...,0,low_priority
1,indicate the language of a code blocktell us a...,0,low_priority
2,use one attribute name for header title namema...,0,low_priority
3,document levels of validation in vldtermas thi...,0,low_priority
4,support puml files in boilerplatessee this com...,0,low_priority
...,...,...,...
247034,twitch client needs to be able to reconnectnee...,1,not_priority
247035,icon is not shown for resource time ranges whe...,1,not_priority
247036,update frameshift or mosaic logos on pdf repor...,1,not_priority
247037,readcount staying at 1 hello when i run contin...,1,not_priority


In [23]:
# Clean dataset with clean text and labels.
# 0 = high priority, 1 = not high priority
file_name = f"csv/clean_low_vs_not_priority.csv"
priority_label_text.to_csv(file_name, index=False)

In [24]:
pri = pd.read_csv(file_name)
pri

,text_clean,labels,class
0,zindex issue using classic blockselecting a li...,0,low_priority
1,indicate the language of a code blocktell us a...,0,low_priority
2,use one attribute name for header title namema...,0,low_priority
3,document levels of validation in vldtermas thi...,0,low_priority
4,support puml files in boilerplatessee this com...,0,low_priority
...,...,...,...
247034,twitch client needs to be able to reconnectnee...,1,not_priority
247035,icon is not shown for resource time ranges whe...,1,not_priority
247036,update frameshift or mosaic logos on pdf repor...,1,not_priority
247037,readcount staying at 1 hello when i run contin...,1,not_priority
